# Iteration 2: elective patients - stage 2

## Plain English Summary

The model also includes an arrival and treatment process for elective patients.  In real life elective surgeries where a patient will require critical care post surgery are cancelled if there are no beds available.  In the model this is simulated in a simple way: elective patients are generated and as they arrive the model logic checks to see if any beds are available.  If no beds are available the elective patient leaves the model immediately - this is called "balking" in queuing theory. If beds are available the elective patient takes one and begins treatment.

This is the first test of more complicated logic for the LLM.




## Notebook imports

In [2]:
from IPython.display import Markdown, display

import numpy as np
import math

## LLM Prompt

In [3]:
display(Markdown("../04_prompts_stage1/inputs/iteration_2.md"))

### Main command

Modify the model to add in an elective admissions treatment process.

Show the full code listing for the model.  Do not exclude any functions or classes.

### general simulation logic

Add a new arrival source to the CCU: Elective surgery

Elective surgery patients are modelled as a separate process from the unplanned admissions, but share the critical care bed resources. 

As an elective patient arrives to the CCU a check is made on the number of critical care beds available.  There are two outcomes from this check

Outcome 1: the number of beds in use is equal to the total number of beds available. In this case the elective patient leaves the model immediately. This is called a "cancelled operation" event and should be reported to the user.

Outcome 2. the number of beds in use is less than the total number of beds available. In this case the elective patient requests a critical care bed, is treated, and is then discharged. Before the bed is released for another patient there is an additional delay for intensive cleaning of the area and the bed.

### simulation inputs

the interarrival time of elective surgery patients has a normal distribution with mean 17.91 and standard deviation 3.16.

the treatment time of elective surgery patients is exponential mean: 57.34

## Date and time of input to LLM

> 1638 30th May 2024

## Observations on the LLM Output

* patient arrival counter addition assignment occurs before yield statement in arrivals. This is fine right now but means patient counts start at 0 - it may need to be changed at some point.
* Patient processes use patient id as an argument instead of referring directly to self.patient_id_counter in the CCU class
* Cancelled operations via balking occurs in the patient process rather than the generator function. It is happening before requesting a resource so seems okay.
* 

# LLM Output 1

In [6]:
import simpy
import numpy as np
from scipy.stats import lognorm

class CCU:
    def __init__(self, env, run_length=12 * 30 * 24):
        self.env = env
        self.run_length = run_length
        self.patient_id_counter = 0
        self.cancelled_operations = 0

        # Set up random number generators for each patient type
        self.rng_accident_emergency = np.random.default_rng(seed=42)
        self.rng_wards = np.random.default_rng(seed=43)
        self.rng_emergency_surgery = np.random.default_rng(seed=44)
        self.rng_other_hospitals = np.random.default_rng(seed=45)
        self.rng_xray_department = np.random.default_rng(seed=46)
        self.rng_elective_surgery = np.random.default_rng(seed=47)

        # Set up critical care bed resource
        self.critical_care_beds = simpy.Resource(env, capacity=24)

        # Lognormal parameters
        self.accident_emergency_lognormal_mu = 128.79
        self.accident_emergency_lognormal_sigma = 267.51
        self.wards_lognormal_mu = 177.89
        self.wards_lognormal_sigma = 276.54
        self.emergency_surgery_lognormal_mu = 140.15
        self.emergency_surgery_lognormal_sigma = 218.02
        self.other_hospitals_lognormal_mu = 212.86
        self.other_hospitals_lognormal_sigma = 457.67
        self.xray_department_lognormal_mu = 87.53
        self.xray_department_lognormal_sigma = 108.67

    def lognormal_to_normal(self, mu, sigma):
        """Convert lognormal parameters to underlying normal distribution"""
        zeta = np.log(mu**2 / np.sqrt(sigma**2 + mu**2))
        sigma_norm = np.sqrt(np.log(sigma**2 / mu**2 + 1))
        mu_norm = zeta
        return mu_norm, sigma_norm

    def accident_emergency_arrivals(self):
        while True:
            patient_id = self.patient_id_counter
            self.patient_id_counter += 1
            yield self.env.timeout(self.rng_accident_emergency.exponential(22.72))
            self.env.process(self.unplanned_admissions_process(patient_id, "Accident and Emergency"))

    def wards_arrivals(self):
        while True:
            patient_id = self.patient_id_counter
            self.patient_id_counter += 1
            yield self.env.timeout(self.rng_wards.exponential(26.0))
            self.env.process(self.unplanned_admissions_process(patient_id, "Wards"))

    def emergency_surgery_arrivals(self):
        while True:
            patient_id = self.patient_id_counter
            self.patient_id_counter += 1
            yield self.env.timeout(self.rng_emergency_surgery.exponential(37.0))
            self.env.process(self.unplanned_admissions_process(patient_id, "Emergency Surgery"))

    def other_hospitals_arrivals(self):
        while True:
            patient_id = self.patient_id_counter
            self.patient_id_counter += 1
            yield self.env.timeout(self.rng_other_hospitals.exponential(47.2))
            self.env.process(self.unplanned_admissions_process(patient_id, "Other Hospitals"))

    def xray_department_arrivals(self):
        while True:
            patient_id = self.patient_id_counter
            self.patient_id_counter += 1
            yield self.env.timeout(self.rng_xray_department.exponential(575.0))
            self.env.process(self.unplanned_admissions_process(patient_id, "X-Ray Department"))

    def elective_surgery_arrivals(self):
        while True:
            patient_id = self.patient_id_counter
            self.patient_id_counter += 1
            yield self.env.timeout(self.rng_elective_surgery.normal(17.91, 3.16))
            self.env.process(self.elective_admissions_process(patient_id))

    def unplanned_admissions_process(self, patient_id, source):
        print(f"Patient {patient_id} from {source} requests a critical care bed at {self.env.now:.2f} hours")
        with self.critical_care_beds.request() as req:
            yield req
            print(f"Patient {patient_id} from {source} admitted to a critical care bed at {self.env.now:.2f} hours")

            # Determine length of stay based on source
            if source == "Accident and Emergency":
                mu, sigma = self.lognormal_to_normal(self.accident_emergency_lognormal_mu, self.accident_emergency_lognormal_sigma)
                length_of_stay = self.rng_accident_emergency.lognormal(mu, sigma)
            elif source == "Wards":
                mu, sigma = self.lognormal_to_normal(self.wards_lognormal_mu, self.wards_lognormal_sigma)
                length_of_stay = self.rng_wards.lognormal(mu, sigma)
            elif source == "Emergency Surgery":
                mu, sigma = self.lognormal_to_normal(self.emergency_surgery_lognormal_mu, self.emergency_surgery_lognormal_sigma)
                length_of_stay = self.rng_emergency_surgery.lognormal(mu, sigma)
            elif source == "Other Hospitals":
                mu, sigma = self.lognormal_to_normal(self.other_hospitals_lognormal_mu, self.other_hospitals_lognormal_sigma)
                length_of_stay = self.rng_other_hospitals.lognormal(mu, sigma)
            else:  # X-Ray Department
                mu, sigma = self.lognormal_to_normal(self.xray_department_lognormal_mu, self.xray_department_lognormal_sigma)
                length_of_stay = self.rng_xray_department.lognormal(mu, sigma)

            yield self.env.timeout(length_of_stay)
            print(f"Patient {patient_id} from {source} discharged from a critical care bed at {self.env.now:.2f} hours")

            # Intensive cleaning
            yield self.env.timeout(5)
            print(f"Intensive cleaning completed for Patient {patient_id} from {source} at {self.env.now:.2f} hours")

    def elective_admissions_process(self, patient_id):
        print(f"Elective Patient {patient_id} requests a critical care bed at {self.env.now:.2f} hours")
        if self.critical_care_beds.count == self.critical_care_beds.capacity:
            print(f"Elective Patient {patient_id} operation cancelled at {self.env.now:.2f} hours due to lack of available beds")
            self.cancelled_operations += 1
        else:
            with self.critical_care_beds.request() as req:
                yield req
                print(f"Elective Patient {patient_id} admitted to a critical care bed at {self.env.now:.2f} hours")
                length_of_stay = self.rng_elective_surgery.exponential(57.34)
                yield self.env.timeout(length_of_stay)
                print(f"Elective Patient {patient_id} discharged from a critical care bed at {self.env.now:.2f} hours")

                # Intensive cleaning
                yield self.env.timeout(5)
                print(f"Intensive cleaning completed for Elective Patient {patient_id} at {self.env.now:.2f} hours")

    def run(self):
        # Start patient arrival processes
        accident_emergency_process = self.env.process(self.accident_emergency_arrivals())
        wards_process = self.env.process(self.wards_arrivals())
        emergency_surgery_process = self.env.process(self.emergency_surgery_arrivals())
        other_hospitals_process = self.env.process(self.other_hospitals_arrivals())
        xray_department_process = self.env.process(self.xray_department_arrivals())
        elective_surgery_process = self.env.process(self.elective_surgery_arrivals())

        self.env.run(until=self.run_length)

        print(f"Total cancelled elective operations: {self.cancelled_operations}")

# Run the simulation
env = simpy.Environment()
ccu = CCU(env)
ccu.run()


Patient 1 from Wards requests a critical care bed at 4.30 hours
Patient 1 from Wards admitted to a critical care bed at 4.30 hours
Patient 6 from Wards requests a critical care bed at 6.54 hours
Patient 6 from Wards admitted to a critical care bed at 6.54 hours
Patient 2 from Emergency Surgery requests a critical care bed at 13.71 hours
Patient 2 from Emergency Surgery admitted to a critical care bed at 13.71 hours
Elective Patient 5 requests a critical care bed at 15.80 hours
Elective Patient 5 admitted to a critical care bed at 15.80 hours
Patient 6 from Wards discharged from a critical care bed at 17.12 hours
Intensive cleaning completed for Patient 6 from Wards at 22.12 hours
Patient 8 from Emergency Surgery requests a critical care bed at 31.66 hours
Patient 8 from Emergency Surgery admitted to a critical care bed at 31.66 hours
Elective Patient 9 requests a critical care bed at 34.02 hours
Elective Patient 9 admitted to a critical care bed at 34.02 hours
Patient 3 from Other Hosp

## Additional prompt required as diff 00-01 found that arrival messages were lost

> Update the code to add an arrival message for each generator function. Add the message after the patient arrival counter addition assignment.

In this updated code, an arrival message is printed after the patient arrival counter addition assignment in each generator function. For example, in the accident_emergency_arrivals function, after assigning the current patient_id_counter value to patient_id and incrementing the counter, a message is printed indicating the patient arrival from the "Accident and Emergency" source along with the current simulation time.
The same pattern is followed for the other generator functions (wards_arrivals, emergency_surgery_arrivals, other_hospitals_arrivals, xray_department_arrivals, and elective_surgery_arrivals).
With these updates, the model will now print an arrival message for each patient, indicating the patient ID, source, and the simulation time when the arrival occurred.

In [7]:
import simpy
import numpy as np
from scipy.stats import lognorm

class CCU:
    def __init__(self, env, run_length=12 * 30 * 24):
        self.env = env
        self.run_length = run_length
        self.patient_id_counter = 0
        self.cancelled_operations = 0

        # Set up random number generators for each patient type
        self.rng_accident_emergency = np.random.default_rng(seed=42)
        self.rng_wards = np.random.default_rng(seed=43)
        self.rng_emergency_surgery = np.random.default_rng(seed=44)
        self.rng_other_hospitals = np.random.default_rng(seed=45)
        self.rng_xray_department = np.random.default_rng(seed=46)
        self.rng_elective_surgery = np.random.default_rng(seed=47)

        # Set up critical care bed resource
        self.critical_care_beds = simpy.Resource(env, capacity=24)

        # Lognormal parameters
        self.accident_emergency_lognormal_mu = 128.79
        self.accident_emergency_lognormal_sigma = 267.51
        self.wards_lognormal_mu = 177.89
        self.wards_lognormal_sigma = 276.54
        self.emergency_surgery_lognormal_mu = 140.15
        self.emergency_surgery_lognormal_sigma = 218.02
        self.other_hospitals_lognormal_mu = 212.86
        self.other_hospitals_lognormal_sigma = 457.67
        self.xray_department_lognormal_mu = 87.53
        self.xray_department_lognormal_sigma = 108.67

    def lognormal_to_normal(self, mu, sigma):
        """Convert lognormal parameters to underlying normal distribution"""
        zeta = np.log(mu**2 / np.sqrt(sigma**2 + mu**2))
        sigma_norm = np.sqrt(np.log(sigma**2 / mu**2 + 1))
        mu_norm = zeta
        return mu_norm, sigma_norm

    def accident_emergency_arrivals(self):
        while True:
            patient_id = self.patient_id_counter
            self.patient_id_counter += 1
            print(f"Patient {patient_id} arrived from Accident and Emergency at {self.env.now:.2f} hours")
            yield self.env.timeout(self.rng_accident_emergency.exponential(22.72))
            self.env.process(self.unplanned_admissions_process(patient_id, "Accident and Emergency"))

    def wards_arrivals(self):
        while True:
            patient_id = self.patient_id_counter
            self.patient_id_counter += 1
            print(f"Patient {patient_id} arrived from the Wards at {self.env.now:.2f} hours")
            yield self.env.timeout(self.rng_wards.exponential(26.0))
            self.env.process(self.unplanned_admissions_process(patient_id, "Wards"))

    def emergency_surgery_arrivals(self):
        while True:
            patient_id = self.patient_id_counter
            self.patient_id_counter += 1
            print(f"Patient {patient_id} arrived from Emergency Surgery at {self.env.now:.2f} hours")
            yield self.env.timeout(self.rng_emergency_surgery.exponential(37.0))
            self.env.process(self.unplanned_admissions_process(patient_id, "Emergency Surgery"))

    def other_hospitals_arrivals(self):
        while True:
            patient_id = self.patient_id_counter
            self.patient_id_counter += 1
            print(f"Patient {patient_id} arrived from Other Hospitals at {self.env.now:.2f} hours")
            yield self.env.timeout(self.rng_other_hospitals.exponential(47.2))
            self.env.process(self.unplanned_admissions_process(patient_id, "Other Hospitals"))

    def xray_department_arrivals(self):
        while True:
            patient_id = self.patient_id_counter
            self.patient_id_counter += 1
            print(f"Patient {patient_id} arrived from the X-Ray Department at {self.env.now:.2f} hours")
            yield self.env.timeout(self.rng_xray_department.exponential(575.0))
            self.env.process(self.unplanned_admissions_process(patient_id, "X-Ray Department"))

    def elective_surgery_arrivals(self):
        while True:
            patient_id = self.patient_id_counter
            self.patient_id_counter += 1
            print(f"Elective Patient {patient_id} arrived at {self.env.now:.2f} hours")
            yield self.env.timeout(self.rng_elective_surgery.normal(17.91, 3.16))
            self.env.process(self.elective_admissions_process(patient_id))

    def unplanned_admissions_process(self, patient_id, source):
        print(f"Patient {patient_id} from {source} requests a critical care bed at {self.env.now:.2f} hours")
        with self.critical_care_beds.request() as req:
            yield req
            print(f"Patient {patient_id} from {source} admitted to a critical care bed at {self.env.now:.2f} hours")

            # Determine length of stay based on source
            if source == "Accident and Emergency":
                mu, sigma = self.lognormal_to_normal(self.accident_emergency_lognormal_mu, self.accident_emergency_lognormal_sigma)
                length_of_stay = self.rng_accident_emergency.lognormal(mu, sigma)
            elif source == "Wards":
                mu, sigma = self.lognormal_to_normal(self.wards_lognormal_mu, self.wards_lognormal_sigma)
                length_of_stay = self.rng_wards.lognormal(mu, sigma)
            elif source == "Emergency Surgery":
                mu, sigma = self.lognormal_to_normal(self.emergency_surgery_lognormal_mu, self.emergency_surgery_lognormal_sigma)
                length_of_stay = self.rng_emergency_surgery.lognormal(mu, sigma)
            elif source == "Other Hospitals":
                mu, sigma = self.lognormal_to_normal(self.other_hospitals_lognormal_mu, self.other_hospitals_lognormal_sigma)
                length_of_stay = self.rng_other_hospitals.lognormal(mu, sigma)
            else:  # X-Ray Department
                mu, sigma = self.lognormal_to_normal(self.xray_department_lognormal_mu, self.xray_department_lognormal_sigma)
                length_of_stay = self.rng_xray_department.lognormal(mu, sigma)

            yield self.env.timeout(length_of_stay)
            print(f"Patient {patient_id} from {source} discharged from a critical care bed at {self.env.now:.2f} hours")

            # Intensive cleaning
            yield self.env.timeout(5)
            print(f"Intensive cleaning completed for Patient {patient_id} from {source} at {self.env.now:.2f} hours")

    def elective_admissions_process(self, patient_id):
        print(f"Elective Patient {patient_id} requests a critical care bed at {self.env.now:.2f} hours")
        if self.critical_care_beds.count == self.critical_care_beds.capacity:
            print(f"Elective Patient {patient_id} operation cancelled at {self.env.now:.2f} hours due to lack of available beds")
            self.cancelled_operations += 1
        else:
            with self.critical_care_beds.request() as req:
                yield req
                print(f"Elective Patient {patient_id} admitted to a critical care bed at {self.env.now:.2f} hours")
                length_of_stay = self.rng_elective_surgery.exponential(57.34)
                yield self.env.timeout(length_of_stay)
                print(f"Elective Patient {patient_id} discharged from a critical care bed at {self.env.now:.2f} hours")

                # Intensive cleaning
                yield self.env.timeout(5)
                print(f"Intensive cleaning completed for Elective Patient {patient_id} at {self.env.now:.2f} hours")

    def run(self):
        # Start patient arrival processes
        accident_emergency_process = self.env.process(self.accident_emergency_arrivals())
        wards_process = self.env.process(self.wards_arrivals())
        emergency_surgery_process = self.env.process(self.emergency_surgery_arrivals())
        other_hospitals_process = self.env.process(self.other_hospitals_arrivals())
        xray_department_process = self.env.process(self.xray_department_arrivals())
        elective_surgery_process = self.env.process(self.elective_surgery_arrivals())

        self.env.run(until=self.run_length)

        print(f"Total cancelled elective operations: {self.cancelled_operations}")

# Run the simulation
env = simpy.Environment()
ccu = CCU(env)
ccu.run()


Patient 0 arrived from Accident and Emergency at 0.00 hours
Patient 1 arrived from the Wards at 0.00 hours
Patient 2 arrived from Emergency Surgery at 0.00 hours
Patient 3 arrived from Other Hospitals at 0.00 hours
Patient 4 arrived from the X-Ray Department at 0.00 hours
Elective Patient 5 arrived at 0.00 hours
Patient 6 arrived from the Wards at 4.30 hours
Patient 1 from Wards requests a critical care bed at 4.30 hours
Patient 1 from Wards admitted to a critical care bed at 4.30 hours
Patient 7 arrived from the Wards at 6.54 hours
Patient 6 from Wards requests a critical care bed at 6.54 hours
Patient 6 from Wards admitted to a critical care bed at 6.54 hours
Patient 8 arrived from Emergency Surgery at 13.71 hours
Patient 2 from Emergency Surgery requests a critical care bed at 13.71 hours
Patient 2 from Emergency Surgery admitted to a critical care bed at 13.71 hours
Elective Patient 9 arrived at 15.80 hours
Elective Patient 5 requests a critical care bed at 15.80 hours
Elective Pat

## Testing

* **Extreme value test 1**: means of stay_distributions set to $M$ a very large number.
    * Expected result: after 24 arrivals queues form and no patients are admitted and all electives are cancelled.
    * Actual result (PASS): after 24 arrivals queues form and no patients are admitted and all electives are cancelled.

* **Extreme value test 2**: All unplanned patient types have have their inter-arrival time set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Elective."
    * Actual result (PASS): The only type of patient to arrive to the model is "Elective."


### Extreme value test 1

PASS

In [8]:
M = 10_000_000

import simpy
import numpy as np
from scipy.stats import lognorm

class CCU:
    def __init__(self, env, run_length=12 * 30 * 24):
        self.env = env
        self.run_length = run_length
        self.patient_id_counter = 0
        self.cancelled_operations = 0

        # Set up random number generators for each patient type
        self.rng_accident_emergency = np.random.default_rng(seed=42)
        self.rng_wards = np.random.default_rng(seed=43)
        self.rng_emergency_surgery = np.random.default_rng(seed=44)
        self.rng_other_hospitals = np.random.default_rng(seed=45)
        self.rng_xray_department = np.random.default_rng(seed=46)
        self.rng_elective_surgery = np.random.default_rng(seed=47)

        # Set up critical care bed resource
        self.critical_care_beds = simpy.Resource(env, capacity=24)

        # Lognormal parameters
        self.accident_emergency_lognormal_mu = M
        self.accident_emergency_lognormal_sigma = 267.51
        self.wards_lognormal_mu = M
        self.wards_lognormal_sigma = 276.54
        self.emergency_surgery_lognormal_mu = M
        self.emergency_surgery_lognormal_sigma = 218.02
        self.other_hospitals_lognormal_mu = M
        self.other_hospitals_lognormal_sigma = 457.67
        self.xray_department_lognormal_mu = M
        self.xray_department_lognormal_sigma = 108.67

    def lognormal_to_normal(self, mu, sigma):
        """Convert lognormal parameters to underlying normal distribution"""
        zeta = np.log(mu**2 / np.sqrt(sigma**2 + mu**2))
        sigma_norm = np.sqrt(np.log(sigma**2 / mu**2 + 1))
        mu_norm = zeta
        return mu_norm, sigma_norm

    def accident_emergency_arrivals(self):
        while True:
            patient_id = self.patient_id_counter
            self.patient_id_counter += 1
            print(f"Patient {patient_id} arrived from Accident and Emergency at {self.env.now:.2f} hours")
            yield self.env.timeout(self.rng_accident_emergency.exponential(22.72))
            self.env.process(self.unplanned_admissions_process(patient_id, "Accident and Emergency"))

    def wards_arrivals(self):
        while True:
            patient_id = self.patient_id_counter
            self.patient_id_counter += 1
            print(f"Patient {patient_id} arrived from the Wards at {self.env.now:.2f} hours")
            yield self.env.timeout(self.rng_wards.exponential(26.0))
            self.env.process(self.unplanned_admissions_process(patient_id, "Wards"))

    def emergency_surgery_arrivals(self):
        while True:
            patient_id = self.patient_id_counter
            self.patient_id_counter += 1
            print(f"Patient {patient_id} arrived from Emergency Surgery at {self.env.now:.2f} hours")
            yield self.env.timeout(self.rng_emergency_surgery.exponential(37.0))
            self.env.process(self.unplanned_admissions_process(patient_id, "Emergency Surgery"))

    def other_hospitals_arrivals(self):
        while True:
            patient_id = self.patient_id_counter
            self.patient_id_counter += 1
            print(f"Patient {patient_id} arrived from Other Hospitals at {self.env.now:.2f} hours")
            yield self.env.timeout(self.rng_other_hospitals.exponential(47.2))
            self.env.process(self.unplanned_admissions_process(patient_id, "Other Hospitals"))

    def xray_department_arrivals(self):
        while True:
            patient_id = self.patient_id_counter
            self.patient_id_counter += 1
            print(f"Patient {patient_id} arrived from the X-Ray Department at {self.env.now:.2f} hours")
            yield self.env.timeout(self.rng_xray_department.exponential(575.0))
            self.env.process(self.unplanned_admissions_process(patient_id, "X-Ray Department"))

    def elective_surgery_arrivals(self):
        while True:
            patient_id = self.patient_id_counter
            self.patient_id_counter += 1
            print(f"Elective Patient {patient_id} arrived at {self.env.now:.2f} hours")
            yield self.env.timeout(self.rng_elective_surgery.normal(17.91, 3.16))
            self.env.process(self.elective_admissions_process(patient_id))

    def unplanned_admissions_process(self, patient_id, source):
        print(f"Patient {patient_id} from {source} requests a critical care bed at {self.env.now:.2f} hours")
        with self.critical_care_beds.request() as req:
            yield req
            print(f"Patient {patient_id} from {source} admitted to a critical care bed at {self.env.now:.2f} hours")

            # Determine length of stay based on source
            if source == "Accident and Emergency":
                mu, sigma = self.lognormal_to_normal(self.accident_emergency_lognormal_mu, self.accident_emergency_lognormal_sigma)
                length_of_stay = self.rng_accident_emergency.lognormal(mu, sigma)
            elif source == "Wards":
                mu, sigma = self.lognormal_to_normal(self.wards_lognormal_mu, self.wards_lognormal_sigma)
                length_of_stay = self.rng_wards.lognormal(mu, sigma)
            elif source == "Emergency Surgery":
                mu, sigma = self.lognormal_to_normal(self.emergency_surgery_lognormal_mu, self.emergency_surgery_lognormal_sigma)
                length_of_stay = self.rng_emergency_surgery.lognormal(mu, sigma)
            elif source == "Other Hospitals":
                mu, sigma = self.lognormal_to_normal(self.other_hospitals_lognormal_mu, self.other_hospitals_lognormal_sigma)
                length_of_stay = self.rng_other_hospitals.lognormal(mu, sigma)
            else:  # X-Ray Department
                mu, sigma = self.lognormal_to_normal(self.xray_department_lognormal_mu, self.xray_department_lognormal_sigma)
                length_of_stay = self.rng_xray_department.lognormal(mu, sigma)

            yield self.env.timeout(length_of_stay)
            print(f"Patient {patient_id} from {source} discharged from a critical care bed at {self.env.now:.2f} hours")

            # Intensive cleaning
            yield self.env.timeout(5)
            print(f"Intensive cleaning completed for Patient {patient_id} from {source} at {self.env.now:.2f} hours")

    def elective_admissions_process(self, patient_id):
        print(f"Elective Patient {patient_id} requests a critical care bed at {self.env.now:.2f} hours")
        if self.critical_care_beds.count == self.critical_care_beds.capacity:
            print(f"Elective Patient {patient_id} operation cancelled at {self.env.now:.2f} hours due to lack of available beds")
            self.cancelled_operations += 1
        else:
            with self.critical_care_beds.request() as req:
                yield req
                print(f"Elective Patient {patient_id} admitted to a critical care bed at {self.env.now:.2f} hours")
                length_of_stay = self.rng_elective_surgery.exponential(57.34)
                yield self.env.timeout(length_of_stay)
                print(f"Elective Patient {patient_id} discharged from a critical care bed at {self.env.now:.2f} hours")

                # Intensive cleaning
                yield self.env.timeout(5)
                print(f"Intensive cleaning completed for Elective Patient {patient_id} at {self.env.now:.2f} hours")

    def run(self):
        # Start patient arrival processes
        accident_emergency_process = self.env.process(self.accident_emergency_arrivals())
        wards_process = self.env.process(self.wards_arrivals())
        emergency_surgery_process = self.env.process(self.emergency_surgery_arrivals())
        other_hospitals_process = self.env.process(self.other_hospitals_arrivals())
        xray_department_process = self.env.process(self.xray_department_arrivals())
        elective_surgery_process = self.env.process(self.elective_surgery_arrivals())

        self.env.run(until=self.run_length)

        print(f"Total cancelled elective operations: {self.cancelled_operations}")

# Run the simulation
env = simpy.Environment()
ccu = CCU(env)
ccu.run()


Patient 0 arrived from Accident and Emergency at 0.00 hours
Patient 1 arrived from the Wards at 0.00 hours
Patient 2 arrived from Emergency Surgery at 0.00 hours
Patient 3 arrived from Other Hospitals at 0.00 hours
Patient 4 arrived from the X-Ray Department at 0.00 hours
Elective Patient 5 arrived at 0.00 hours
Patient 6 arrived from the Wards at 4.30 hours
Patient 1 from Wards requests a critical care bed at 4.30 hours
Patient 1 from Wards admitted to a critical care bed at 4.30 hours
Patient 7 arrived from the Wards at 6.54 hours
Patient 6 from Wards requests a critical care bed at 6.54 hours
Patient 6 from Wards admitted to a critical care bed at 6.54 hours
Patient 8 arrived from Emergency Surgery at 13.71 hours
Patient 2 from Emergency Surgery requests a critical care bed at 13.71 hours
Patient 2 from Emergency Surgery admitted to a critical care bed at 13.71 hours
Elective Patient 9 arrived at 15.80 hours
Elective Patient 5 requests a critical care bed at 15.80 hours
Elective Pat

### Extreme value test 2

PASS.

M = 10_000_000

In [9]:
import simpy
import numpy as np
from scipy.stats import lognorm

class CCU:
    def __init__(self, env, run_length=12 * 30 * 24):
        self.env = env
        self.run_length = run_length
        self.patient_id_counter = 0
        self.cancelled_operations = 0

        # Set up random number generators for each patient type
        self.rng_accident_emergency = np.random.default_rng(seed=42)
        self.rng_wards = np.random.default_rng(seed=43)
        self.rng_emergency_surgery = np.random.default_rng(seed=44)
        self.rng_other_hospitals = np.random.default_rng(seed=45)
        self.rng_xray_department = np.random.default_rng(seed=46)
        self.rng_elective_surgery = np.random.default_rng(seed=47)

        # Set up critical care bed resource
        self.critical_care_beds = simpy.Resource(env, capacity=24)

        # Lognormal parameters
        self.accident_emergency_lognormal_mu = 128.79
        self.accident_emergency_lognormal_sigma = 267.51
        self.wards_lognormal_mu = 177.89
        self.wards_lognormal_sigma = 276.54
        self.emergency_surgery_lognormal_mu = 140.15
        self.emergency_surgery_lognormal_sigma = 218.02
        self.other_hospitals_lognormal_mu = 212.86
        self.other_hospitals_lognormal_sigma = 457.67
        self.xray_department_lognormal_mu = 87.53
        self.xray_department_lognormal_sigma = 108.67

    def lognormal_to_normal(self, mu, sigma):
        """Convert lognormal parameters to underlying normal distribution"""
        zeta = np.log(mu**2 / np.sqrt(sigma**2 + mu**2))
        sigma_norm = np.sqrt(np.log(sigma**2 / mu**2 + 1))
        mu_norm = zeta
        return mu_norm, sigma_norm

    def accident_emergency_arrivals(self):
        while True:
            patient_id = self.patient_id_counter
            self.patient_id_counter += 1
            print(f"Patient {patient_id} arrived from Accident and Emergency at {self.env.now:.2f} hours")
            yield self.env.timeout(self.rng_accident_emergency.exponential(M))
            self.env.process(self.unplanned_admissions_process(patient_id, "Accident and Emergency"))

    def wards_arrivals(self):
        while True:
            patient_id = self.patient_id_counter
            self.patient_id_counter += 1
            print(f"Patient {patient_id} arrived from the Wards at {self.env.now:.2f} hours")
            yield self.env.timeout(self.rng_wards.exponential(M))
            self.env.process(self.unplanned_admissions_process(patient_id, "Wards"))

    def emergency_surgery_arrivals(self):
        while True:
            patient_id = self.patient_id_counter
            self.patient_id_counter += 1
            print(f"Patient {patient_id} arrived from Emergency Surgery at {self.env.now:.2f} hours")
            yield self.env.timeout(self.rng_emergency_surgery.exponential(M))
            self.env.process(self.unplanned_admissions_process(patient_id, "Emergency Surgery"))

    def other_hospitals_arrivals(self):
        while True:
            patient_id = self.patient_id_counter
            self.patient_id_counter += 1
            print(f"Patient {patient_id} arrived from Other Hospitals at {self.env.now:.2f} hours")
            yield self.env.timeout(self.rng_other_hospitals.exponential(M))
            self.env.process(self.unplanned_admissions_process(patient_id, "Other Hospitals"))

    def xray_department_arrivals(self):
        while True:
            patient_id = self.patient_id_counter
            self.patient_id_counter += 1
            print(f"Patient {patient_id} arrived from the X-Ray Department at {self.env.now:.2f} hours")
            yield self.env.timeout(self.rng_xray_department.exponential(M))
            self.env.process(self.unplanned_admissions_process(patient_id, "X-Ray Department"))

    def elective_surgery_arrivals(self):
        while True:
            patient_id = self.patient_id_counter
            self.patient_id_counter += 1
            print(f"Elective Patient {patient_id} arrived at {self.env.now:.2f} hours")
            yield self.env.timeout(self.rng_elective_surgery.normal(17.91, 3.16))
            self.env.process(self.elective_admissions_process(patient_id))

    def unplanned_admissions_process(self, patient_id, source):
        print(f"Patient {patient_id} from {source} requests a critical care bed at {self.env.now:.2f} hours")
        with self.critical_care_beds.request() as req:
            yield req
            print(f"Patient {patient_id} from {source} admitted to a critical care bed at {self.env.now:.2f} hours")

            # Determine length of stay based on source
            if source == "Accident and Emergency":
                mu, sigma = self.lognormal_to_normal(self.accident_emergency_lognormal_mu, self.accident_emergency_lognormal_sigma)
                length_of_stay = self.rng_accident_emergency.lognormal(mu, sigma)
            elif source == "Wards":
                mu, sigma = self.lognormal_to_normal(self.wards_lognormal_mu, self.wards_lognormal_sigma)
                length_of_stay = self.rng_wards.lognormal(mu, sigma)
            elif source == "Emergency Surgery":
                mu, sigma = self.lognormal_to_normal(self.emergency_surgery_lognormal_mu, self.emergency_surgery_lognormal_sigma)
                length_of_stay = self.rng_emergency_surgery.lognormal(mu, sigma)
            elif source == "Other Hospitals":
                mu, sigma = self.lognormal_to_normal(self.other_hospitals_lognormal_mu, self.other_hospitals_lognormal_sigma)
                length_of_stay = self.rng_other_hospitals.lognormal(mu, sigma)
            else:  # X-Ray Department
                mu, sigma = self.lognormal_to_normal(self.xray_department_lognormal_mu, self.xray_department_lognormal_sigma)
                length_of_stay = self.rng_xray_department.lognormal(mu, sigma)

            yield self.env.timeout(length_of_stay)
            print(f"Patient {patient_id} from {source} discharged from a critical care bed at {self.env.now:.2f} hours")

            # Intensive cleaning
            yield self.env.timeout(5)
            print(f"Intensive cleaning completed for Patient {patient_id} from {source} at {self.env.now:.2f} hours")

    def elective_admissions_process(self, patient_id):
        print(f"Elective Patient {patient_id} requests a critical care bed at {self.env.now:.2f} hours")
        if self.critical_care_beds.count == self.critical_care_beds.capacity:
            print(f"Elective Patient {patient_id} operation cancelled at {self.env.now:.2f} hours due to lack of available beds")
            self.cancelled_operations += 1
        else:
            with self.critical_care_beds.request() as req:
                yield req
                print(f"Elective Patient {patient_id} admitted to a critical care bed at {self.env.now:.2f} hours")
                length_of_stay = self.rng_elective_surgery.exponential(57.34)
                yield self.env.timeout(length_of_stay)
                print(f"Elective Patient {patient_id} discharged from a critical care bed at {self.env.now:.2f} hours")

                # Intensive cleaning
                yield self.env.timeout(5)
                print(f"Intensive cleaning completed for Elective Patient {patient_id} at {self.env.now:.2f} hours")

    def run(self):
        # Start patient arrival processes
        accident_emergency_process = self.env.process(self.accident_emergency_arrivals())
        wards_process = self.env.process(self.wards_arrivals())
        emergency_surgery_process = self.env.process(self.emergency_surgery_arrivals())
        other_hospitals_process = self.env.process(self.other_hospitals_arrivals())
        xray_department_process = self.env.process(self.xray_department_arrivals())
        elective_surgery_process = self.env.process(self.elective_surgery_arrivals())

        self.env.run(until=self.run_length)

        print(f"Total cancelled elective operations: {self.cancelled_operations}")

# Run the simulation
env = simpy.Environment()
ccu = CCU(env)
ccu.run()


Patient 0 arrived from Accident and Emergency at 0.00 hours
Patient 1 arrived from the Wards at 0.00 hours
Patient 2 arrived from Emergency Surgery at 0.00 hours
Patient 3 arrived from Other Hospitals at 0.00 hours
Patient 4 arrived from the X-Ray Department at 0.00 hours
Elective Patient 5 arrived at 0.00 hours
Elective Patient 6 arrived at 15.80 hours
Elective Patient 5 requests a critical care bed at 15.80 hours
Elective Patient 5 admitted to a critical care bed at 15.80 hours
Elective Patient 7 arrived at 34.02 hours
Elective Patient 6 requests a critical care bed at 34.02 hours
Elective Patient 6 admitted to a critical care bed at 34.02 hours
Elective Patient 8 arrived at 45.80 hours
Elective Patient 7 requests a critical care bed at 45.80 hours
Elective Patient 7 admitted to a critical care bed at 45.80 hours
Elective Patient 9 arrived at 61.18 hours
Elective Patient 8 requests a critical care bed at 61.18 hours
Elective Patient 8 admitted to a critical care bed at 61.18 hours
El

In [6]:
import simpy
import numpy as np

# Define the critical care unit (CCU) model with unplanned admissions and elective surgery
class CCUModel:
    def __init__(self, env, run_length=12*30*24):  # Default run length is 12 months
        self.env = env
        self.run_length = run_length
        self.sources = ['Accident and Emergency', 'the Wards', 'Emergency surgery', 'other hospitals', 'the X-Ray department', 'Elective surgery']
        self.interarrival_distributions = [22.72, 26.0, 37.0, 47.2, 575.0, 17.91]
        self.stay_distributions = [(128.79, 267.51), (177.89, 276.54), (140.15, 218.02), (212.86, 457.67), (87.53, 108.67), (57.34)]
        self.patient_count = 0
        self.critical_care_beds = simpy.Resource(env, capacity=24)

    # Generator function for patient arrival from Elective surgery
    def patient_arrival_elective_surgery(self):
        while True:
            yield self.env.timeout(np.random.normal(17.91, 3.16))
            self.patient_count += 1
            print(f"Elective surgery patient {self.patient_count} arrived at time {self.env.now}")
            if len(self.critical_care_beds.users) == self.critical_care_beds.capacity:
                print(f"Elective surgery for patient {self.patient_count} cancelled due to no available critical care beds at time {self.env.now}")
            else:
                self.env.process(self.elective_surgery_process(self.stay_distributions[5]))

    # Elective surgery process
    def elective_surgery_process(self, stay_distribution):
        with self.critical_care_beds.request() as req:
            yield req
            print(f"Elective surgery patient {self.patient_count} admitted to critical care bed at time {self.env.now}")
            treatment_time = np.random.exponential(57.34)
            yield self.env.timeout(treatment_time)
            print(f"Elective surgery patient {self.patient_count} discharged from critical care bed at time {self.env.now}")
            yield self.env.timeout(5)  # Intensive cleaning time
            print(f"Critical care bed is available for next patient at time {self.env.now}")

    # Generator function for patient arrival from Accident and Emergency
    def patient_arrival_AE(self):
        while True:
            yield self.env.timeout(np.random.exponential(M))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from Accident and Emergency at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.stay_distributions[0]))

    # Generator function for patient arrival from the Wards
    def patient_arrival_wards(self):
        while True:
            yield self.env.timeout(np.random.exponential(M))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from the Wards at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.stay_distributions[1]))

    # Generator function for patient arrival from Emergency surgery
    def patient_arrival_surgery(self):
        while True:
            yield self.env.timeout(np.random.exponential(M))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from Emergency surgery at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.stay_distributions[2]))

    # Generator function for patient arrival from other hospitals
    def patient_arrival_other_hospitals(self):
        while True:
            yield self.env.timeout(np.random.exponential(M))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from other hospitals at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.stay_distributions[3]))

    # Generator function for patient arrival from the X-Ray department
    def patient_arrival_X_ray(self):
        while True:
            yield self.env.timeout(np.random.exponential(M))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from the X-Ray department at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.stay_distributions[4]))

    # Unplanned admissions treatment process
    def unplanned_admission(self, stay_distribution):
        with self.critical_care_beds.request() as req:
            yield req
            print(f"Patient {self.patient_count} admitted to critical care bed at time {self.env.now}")
            treatment_time = np.random.lognormal(np.log(stay_distribution[0]) - 0.5 * np.log(1 + (stay_distribution[1] / stay_distribution[0])**2),
                                                 np.sqrt(np.log(1 + (stay_distribution[1] / stay_distribution[0])**2)))
            yield self.env.timeout(treatment_time)
            print(f"Patient {self.patient_count} discharged from critical care bed at time {self.env.now}")
            yield self.env.timeout(5)  # Intensive cleaning time
            print(f"Critical care bed is available for next patient at time {self.env.now}")

    # Run the simulation
    def run(self):
        env.process(self.patient_arrival_AE())
        env.process(self.patient_arrival_wards())
        env.process(self.patient_arrival_surgery())
        env.process(self.patient_arrival_other_hospitals())
        env.process(self.patient_arrival_X_ray())
        env.process(self.patient_arrival_elective_surgery())
        env.run(until=self.run_length)

# Create a SimPy environment
env = simpy.Environment()

# Create an instance of the CCU model and run the simulation
ccu_model = CCUModel(env)
ccu_model.run()


Elective surgery patient 1 arrived at time 13.12468794142434
Elective surgery patient 1 admitted to critical care bed at time 13.12468794142434
Elective surgery patient 2 arrived at time 27.81240390689647
Elective surgery patient 2 admitted to critical care bed at time 27.81240390689647
Elective surgery patient 3 arrived at time 42.896561217666935
Elective surgery patient 3 admitted to critical care bed at time 42.896561217666935
Elective surgery patient 3 discharged from critical care bed at time 43.602887287672026
Critical care bed is available for next patient at time 48.602887287672026
Elective surgery patient 4 arrived at time 64.24764394413164
Elective surgery patient 4 admitted to critical care bed at time 64.24764394413164
Elective surgery patient 4 discharged from critical care bed at time 72.19770511657947
Elective surgery patient 4 discharged from critical care bed at time 74.87708509476944
Critical care bed is available for next patient at time 77.19770511657947
Critical ca